In [1]:
import subprocess, re, csv, os, shutil
from datetime import datetime

In [2]:
subprocess.run(["gphoto2", "--auto-detect"])

CompletedProcess(args=['gphoto2', '--auto-detect'], returncode=0)

## ポート取得

In [2]:
def get_lines(cmd):
    '''
    :param cmd: str 実行するコマンド.
    :rtype: generator
    :return: 標準出力 (行毎).
    '''
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    while True:
        line = proc.stdout.readline()
        if line:
            yield line

        if not line and proc.poll() is not None:
            break

In [3]:
def get_list_ports():
    output = get_lines("gphoto2 --auto-detect")
    list_ports = []
    for line in output:
        list_ports.append(line.decode('utf-8'))
    list_ports=list_ports[2:]
    list_ports = [re.split("\s\s+", port)[1] for port in list_ports]
    return list_ports

In [6]:
list_ports = get_list_ports()

In [7]:
list_ports

['usb:020,017', 'usb:020,019']

In [8]:
len(list_ports)

2

## シリアルナンバー取得

In [9]:
PTN_SERIALNUM = re.compile("Current:\s+(?P<serialnum>\d+)")
def get_serialnum(port):
    output = get_lines("gphoto2 --port " + port + " --get-config eosserialnumber")
    lines = [line for line in output]
    serialnum = lines[2].decode('utf-8')
    m = PTN_SERIALNUM.match(serialnum)
    serialnum = m.group("serialnum")
    return serialnum

def get_list_serialnumber(list_ports):
    list_serialnum = []
    for port in list_ports:
        serialnum = get_serialnum(port)
        list_serialnum.append(serialnum)
    return list_serialnum

In [10]:
list_serialnum = get_list_serialnumber(list_ports)

In [11]:
list_serialnum

['011074017969', '011074017959']

## 画像の取得

In [12]:
list_camera_numbers = []
with open("./serialnum.csv", "r") as f:
    reader = csv.reader(f, delimiter=',' ,skipinitialspace=True)
    for row in reader:
        list_camera_numbers.append(row)

In [13]:
def genListCameras(list_camera_numbers, list_ports):
    list_cameras = []
    for port in list_ports:
        serialnum = get_serialnum(port)
        for cameraNum in list_camera_numbers:
            if cameraNum[1] == serialnum:
                list_cameras.append([cameraNum[0], cameraNum[1], port])
    list_cameras.sort(key=lambda x:x[0])
    return list_cameras

In [14]:
list_cameras = genListCameras(list_camera_numbers, list_ports)

In [15]:
list_cameras

[['t03', '011074017969', 'usb:020,017'],
 ['t04', '011074017959', 'usb:020,019']]

In [16]:
PTN_IMAGES_IN_CAMERA = re.compile("^#(?P<num>\d+)\s+(?P<filename>[^\s]+)\s+")
def get_list_files(port):
    list_file = []
    output = get_lines("gphoto2 --port " + port + " --list-files")
    lines = [line.decode('utf-8') for line in output]
    for line in lines:
#         print(line)
        m = PTN_IMAGES_IN_CAMERA.match(line)
        if m is not None:
            list_file.append([m.group("num"), m.group("filename")])
    return list_file

In [17]:
get_list_files(list_cameras[0][2])

[['1', 'IMG_0001.JPG'], ['2', 'IMG_0002.JPG'], ['3', 'IMG_0003.JPG']]

In [18]:
def get_all_images(list_cameras, outputPath, overwrite=False):
    cwd = os.getcwd()
    wd = os.path.normpath(os.path.join(cwd, outputPath))
    if not os.path.exists(wd):
        os.mkdir(wd)
    os.chdir(wd)
    for camera in list_cameras:
        camDirPath = os.path.join(wd,camera[0])
        if not os.path.exists(camDirPath):
            os.mkdir(camDirPath)
        else:
            shutil.rmtree(camDirPath)
            os.mkdir(camDirPath)
        print(camera)
        subprocess.run(["gphoto2", "--port", camera[2], "--get-all-files", "--filename", camera[0] +"/%04n.%C"])
    os.chdir(cwd)

In [19]:
get_all_images(list_cameras, "./test")
# バッテリー切れに注意．もし接続が切れていれば，その台数だけ最初に取得されるカメラからダウンロードされる．

['t03', '011074017969', 'usb:020,017']
['t04', '011074017959', 'usb:020,019']


# 画像フォーマットの設定

In [20]:
def set_imageformat(port, formatNum):
    output = get_lines("gphoto2 --port " + port + " --set-config imageformat="+str(formatNum))
    return [line for line in output]

def set_allCameras_imageformat(list_ports, formatNum):
    for port in list_ports:
        set_imageformat(port, formatNum)
    return 0

# Choice: 0 Large Fine JPEG
# Choice: 1 Large Normal JPEG
# Choice: 2 Medium Fine JPEG
# Choice: 3 Medium Normal JPEG
# Choice: 4 Small Fine JPEG
# Choice: 5 Small Normal JPEG
# Choice: 6 Smaller JPEG
# Choice: 7 Tiny JPEG
# Choice: 8 RAW + Large Fine JPEG
# Choice: 9 RAW

In [21]:
set_allCameras_imageformat(list_ports, 9)

0

## SDカードのフォーマット（データの削除）

In [22]:
def delete_all_files(port):
    output = get_lines("gphoto2 --port " + port + " --delete-all-files --recurse")
    return [line for line in output]

def delete_all_files_inAllCameras(list_ports):
    for port in list_ports:
        delete_all_files(port)
    return 0    

In [23]:
delete_all_files_inAllCameras(list_ports)

0

## 時刻の同期

In [24]:
PTN_DATETIME = re.compile("^Current:\s+(?P<datetime>\d+)")
def check_clocks(list_ports):
    for port in list_ports:
        output = get_lines("gphoto2 --port " + port + " --get-config datetime")
        lines = [line for line in output]
        m = PTN_DATETIME.match(lines[2].decode("utf-8"))
        dtStr = m.group("datetime")
        dt = datetime.fromtimestamp(int(dtStr))
        print(dt.isoformat())
def set_clocks_now(list_ports):
    for port in list_ports:
        output = get_lines("gphoto2 --port " + port + " --set-config datetime=now")
        lines = [line for line in output]

In [27]:
check_clocks(list_ports)

2017-06-18T22:48:07
2017-06-18T22:48:07


In [26]:
set_clocks_now(list_ports)

In [247]:
os.path.normpath(os.path.join(os.getcwd(),"./test" ))

'/Users/noshita/working_dir/cameraControl/test'

In [66]:
lines = [line for line in output]

In [67]:
lines

[b'Label: Serial Number\n', b'Type: TEXT\n', b'Current: 031074011540\n']